In [1]:
import re
import sys,os
sys.path.append('../../../classifier/models/time_pattern/')
from time_pattern import TimePattern

In [2]:
tp = TimePattern()

Time Zone is set from ENV: Asia/Shanghai


In [40]:
s1 = '我可以先还5万2，但要100天后才能还清'
s2 = '我可以先还52000，但要100天后才能还清'
s3 = '我可以先还52，000.23， 但要23点15分后才能还清'
s4 = '我可以先还五万6千零二十'
s5 = '但要100天后才能还清'

In [41]:
clean = s3
clean = re.sub(r',|，| ','',clean)
# remove time

In [42]:
number = re.findall(r'([一二三四五六七八九零十百千万亿]+|[0-9]+[,]*[0-9]+.[0-9]+)',clean)
number

['52000.23', '23点15']

In [44]:
number = re.findall(r'(?:还)[一二三四五六七八九零十百千万亿兆0-9点.]+',clean)
number

['52000.23', '23点15']

In [48]:
tp.process('明天早上11点')

[{'pattern': '明天早上11点',
  'time': datetime.datetime(2018, 8, 9, 11, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>),
  'gapS': 44349.645987,
  'gapH': 12.319346107500001}]

In [63]:
reg = '今天|明天(?:(?:上午)|(?:早上)|(?:早晨)|(?:凌晨))[1一]点'
re.findall(reg,'明天上午一点')

['明天上午一点']